In [2]:
import PyPDF2
import os
import fitz  # PyMuPDF for PDF text extraction
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
import pinecone
from tqdm.autonotebook import tqdm
from dotenv import load_dotenv,find_dotenv
from langchain.vectorstores import Pinecone
from tqdm.autonotebook import tqdm

c:\Users\prasu\miniconda3\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
def extract_text_from_pdf(pdf_path):
    # Extract text from the PDF using PyMuPDF (Fitz)
    doc = fitz.open(pdf_path)
    text = ''
    for page_num in range(doc.page_count):
        page = doc[page_num]
        text += page.get_text()
    doc.close()
    return text

# Replace with the path to your PDF files directory
pdf_files_directory = 'Data/pdf'

# List all PDF files in the directory
pdf_files = [os.path.join(pdf_files_directory, filename) for filename in os.listdir(pdf_files_directory) if filename.endswith('.pdf')]


# Iterate through each PDF file
for pdf_file in pdf_files:
    # Extract text from the PDF
    pdf_text = extract_text_from_pdf(pdf_file)
print(pdf_text)



10/13/23, 12:18 PM
MS-BAIS Advising FAQ: ORG04655 : MS-BAIS Current Students
https://usflearn.instructure.com/courses/1794296/pages/ms-bais-advising-faq?module_item_id=28098679
1/6
MS-BAIS Advising FAQ
How does the MS-BAIS program work?
The program requires a minimum of 33 credits, which must include 5 core classes, 3 MS-BAIS
electives and 3 additional electives from our department or other departments if they are 6000 level
courses relevant to the major. Most of these classes are 3 credits each. In addition, you can also take
MS-BAIS internships and independent study, which are 1-2 credit courses and those courses will also
count toward your 33 credit degree requirements.
Before registering for any out-of-department electives, please consult Dr. Johannes Reichgelt to
make sure that this course would count toward your degree requirement. For non-departmental
courses, you will need a permit from that department.
 
What are the different core and elective course options that I have for m

In [4]:


# Import utility for splitting up texts and split up the explanation given above into document chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 0,
)

In [5]:
# Split the text into chunks using RecursiveCharacterTextSplitter
chunks = text_splitter.create_documents([pdf_text])
 # Print or process the chunks as needed
for chunk in chunks:
    print(chunk)

page_content='10/13/23, 12:18 PM\nMS-BAIS Advising FAQ: ORG04655 : MS-BAIS Current Students\nhttps://usflearn.instructure.com/courses/1794296/pages/ms-bais-advising-faq?module_item_id=28098679\n1/6\nMS-BAIS Advising FAQ\nHow does the MS-BAIS program work?\nThe program requires a minimum of 33 credits, which must include 5 core classes, 3 MS-BAIS\nelectives and 3 additional electives from our department or other departments if they are 6000 level'
page_content='courses relevant to the major. Most of these classes are 3 credits each. In addition, you can also take\nMS-BAIS internships and independent study, which are 1-2 credit courses and those courses will also\ncount toward your 33 credit degree requirements.\nBefore registering for any out-of-department electives, please consult Dr. Johannes Reichgelt to\nmake sure that this course would count toward your degree requirement. For non-departmental\ncourses, you will need a permit from that department.'
page_content='What are the differ

In [7]:
embeddings = OpenAIEmbeddings(deployment="text-embedding-ada-002")

In [8]:
# Import and initialize Pinecone client
# You will need to create a Pinecone account and create an API key and environment variable, see www.pinecone.io


pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),
    environment=os.getenv('PINECONE_ENV')
)

In [10]:
# print(os.getenv('PINECONE_API_KEY'))

In [11]:
indexes_info = pinecone.list_indexes()
print("Indexes:", indexes_info)

Indexes: []


In [12]:
# pinecone.init(api_key=os.getenv('PINECONE_API_KEY'), environment=os.getenv('PINECONE_ENV'))
pinecone.create_index("langchain-demo-index", dimension=1536)

In [13]:
# Upload vectors to Pinecone
# You need first create an index in Pinecone, see www.pinecone.io - set it to 1536 dimensions

index_name = "langchain-demo-index"
search = Pinecone.from_documents(chunks, embeddings, index_name=index_name) # load all text chunks into Pinecone with the associated embeddings

In [16]:
# Do a simple vector similarity search
from pprint import pprint
query = "prerequesite for system analysis and design?"
result = search.similarity_search(query)

pprint(result)

[Document(page_content='What are the different core and elective course options that I have for my MS-BAIS degree?\nMS-BAIS core classes:\n1. ISM 6124: Advanced Systems Analysis & Design (Prerequisite: Systems Analysis & Design or\nequivalent professional knowledge)\n2. ISM 6218: Advanced Database Management (Prerequisite: Database Design/Management or\nequivalent professional knowledge)\n3. ISM 6225: Distributed Information Systems (Prerequisite: Object-Oriented Programming or\nequivalent professional knowledge)'),
 Document(page_content='4. QMB 6304: Analytical Methods for Business (Prerequisite: One prior course on inferential\nstatistics at undergraduate or graduate level)\n5. ISM 6155: Enterprise Information Systems Management (MS-BAIS capstone)\nNOTE: In order not to jeopardize your chances to graduate on time, it is crucial that you complete the\ncore courses as early as possible.  We have had a number students who did not take core courses'),
 Document(page_content='3. ISM 6137